In [3]:
import pandas as pd
import openai

In [4]:
with open('key.txt') as f:
    openai.api_key = f.readline()

def toVector(input_text):
    raw_result = openai.Embedding.create(input =input_text, model="text-embedding-ada-002")
    vectors = raw_result["data"][0]["embedding"]
    return vectors

def distance(inputA, inputB):
    dist_sum = 0
    for dimension in range(len(inputA)):
        dist_sum += (inputA[dimension] - inputB[dimension])**2
    return dist_sum**0.5

sentencesList = ["I enjoy walking with my cute dog.","My dog is very happy","I want to play with my cat","I like running"]
vectorsList = []
for i in sentencesList:   
    vectorsList.append(toVector(i))

user_input="Me and my dog are running right now."

user_vector = toVector(user_input)

for i in vectorsList:
    d = distance(user_vector,i)
    print(d)
 


In [32]:
dataNews = pd.read_json("news.json",lines=True,orient="records")
dataNews.head()

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22


In [35]:
dataNews["vector"] = dataNews["short_description"].apply(toVector)
dataNews[["vector","short_description"]].head()

In [41]:
with open("credPinecode.txt") as f:
    info = f.readline().split(";")
    pinecone_env = info[0]
    pinecone_key = info[1]

In [45]:
import pinecone
pinecone.init(api_key=pinecone_key, environment=pinecone_env)

In [ ]:
if "news" in pinecone.list_indexes():
    print("Index already exists")
else:
    dimension = len(dataNews.iloc[0]["vector"])
    pinecone.create_index("news",dimension=dimension,metric="cosine")

In [66]:
selectedIndex = pinecone.Index("news")
selectedIndex.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 50}},
 'total_vector_count': 50}

In [64]:
upsertData = []
for index, row in dataNews.iterrows():
    upsertData.append((str(index),row["vector"]))

In [65]:
selectedIndex.upsert(upsertData)

{'upserted_count': 50}

In [77]:
user_input = "I want to know which books are releasing this week?"
user_vector = toVector(user_input)

result = selectedIndex.query(vector=user_vector,top_k=3,include_values=False)
id = int(result["matches"][0]["id"])
print(result)

{'matches': [{'id': '45', 'score': 0.753737, 'values': []},
             {'id': '32', 'score': 0.741754353, 'values': []},
             {'id': '12', 'score': 0.729336321, 'values': []}],
 'namespace': ''}


In [81]:
dataNews.iloc[45]["short_description"]

'The Colombian-American author\'s new book "Lightlark" was rejected numerous times — until she went directly to her readers for support.'

45